In [1]:
import tqdm
import numpy as np
import utils

# API Setup

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

# Dataset

We've abstracted away the code from the previous notebooks to focus on the concepts from this notebook.

In [3]:
data = utils.load_data(sample_size=100)

Repo card metadata block was not found. Setting CardData to empty.


# Preprocessing
We've abstracted away the code from the previous notebooks to focus on the concepts from this notebook.

In [4]:
documents = utils.preprocess_data(data)

100%|██████████| 100/100 [00:00<00:00, 3909.46it/s]


# Chunking
We've abstracted away the code from the previous notebooks to focus on the concepts from this notebook.

In [5]:
nodes = utils.chunk_documents(documents)

Parsing nodes:   0%|          | 0/100 [00:00<?, ?it/s]

Documents before chunking: 100
Documents after chunking: 837


# Embedding

For our embedding model we use the [BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5) model. This is an MIT license model which is, as of June 1st 2024, 44th on the [MTEB Retrieval leaderboard](https://huggingface.co/spaces/mteb/leaderboard). While there are better models available (as seen on the leaderboard), we choose this model for the demo as it's very small (33M parameters / ~120MB) and hence, very fast. The generated embeddings are 384-dimensional. 

Embedding is usually performed while indexing, so we compute embeddings over our dataset in the next notebook.

In [6]:
# Load embedding model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", embed_batch_size=32)

/Users/akashsaravanan/Library/Caches/pypoetry/virtualenvs/genai-bootcamp-4wh1UwyX-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Example of how to get an embedding and what it looks like
embedding = np.array(embedding_model.get_text_embedding(nodes[0].text))
print(embedding.shape)
embedding[:20]

(384,)


array([-0.02077559, -0.03510703, -0.03307754,  0.02809924,  0.00985101,
       -0.03071311, -0.0626043 ,  0.03460966, -0.0053006 ,  0.00643425,
       -0.03491058, -0.01346424,  0.00973109,  0.01440061,  0.07204092,
       -0.05149863,  0.00747196, -0.01969269, -0.03814155,  0.00673186])